# LightGBM Regressor - Dự đoán Tuổi thọ Trung bình

## Mục tiêu:
- Xây dựng mô hình LightGBM Regressor để dự đoán tuổi thọ trung bình
- Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`
- Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation
- Đánh giá mô hình trên tập validation và test
- Lưu mô hình đã huấn luyện và trực quan hóa kết quả

## Giới thiệu
LightGBM (Light Gradient Boosting Machine) là một framework gradient boosting được phát triển bởi Microsoft, tối ưu hóa cho tốc độ và hiệu suất cao. LightGBM sử dụng các kỹ thuật đột phá để cải thiện tốc độ huấn luyện và giảm sử dụng bộ nhớ.

### **Ưu điểm:**
- Tốc độ huấn luyện cực nhanh (nhanh hơn XGBoost 10-20 lần)
- Hiệu suất cao, thường tương đương hoặc tốt hơn XGBoost
- Xử lý tốt dữ liệu lớn và nhiều chiều

### **Nhược điểm:**
- Dễ bị overfitting với dữ liệu nhỏ.
- Nhạy cảm với siêu tham số.


## Bước 1 - Import các thư viện cần thiết

In [ ]:
# Thư viện cơ bản
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from datetime import datetime

# Thư viện LightGBM
import lightgbm as lgb
from lightgbm import LGBMRegressor

# Thư viện sklearn cho đánh giá
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Thư viện để lưu mô hình
import joblib

# Cài đặt
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Đã import thành công tất cả thư viện!")
print(f"LightGBM version: {lgb.__version__}")

## Bước 2 - Đọc dữ liệu đã tiền xử lý

Đọc dữ liệu từ các file CSV đã được tiền xử lý và chia sẵn thành train/validation/test.

In [ ]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')
val_df = pd.read_csv('../data/processed/val.csv')
test_df = pd.read_csv('../data/processed/test.csv')

print("="*60)
print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập Train:      {train_df.shape}")
print(f"Kích thước tập Validation: {val_df.shape}")
print(f"Kích thước tập Test:       {test_df.shape}")
print("="*60)

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập Train:")
train_df.head()

## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`, và `year`.

In [ ]:
# Định nghĩa các cột đặc trưng (loại bỏ cột không cần thiết)
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code', 'year']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

X_val = val_df[feature_cols]
y_val = val_df['life_expectancy']

X_test = test_df[feature_cols]
y_test = test_df['life_expectancy']

print("="*60)
print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")
print(f"Kích thước X_val:   {X_val.shape}")
print(f"Kích thước y_val:   {y_val.shape}")
print(f"Kích thước X_test:  {X_test.shape}")
print(f"Kích thước y_test:  {y_test.shape}")
print("="*60)

## Bước 4 - Xây dựng mô hình LightGBM cơ bản

Trước tiên, xây dựng một mô hình LightGBM cơ bản với tham số mặc định để làm baseline.

In [ ]:
# Tạo mô hình LightGBM với tham số mặc định
lgbm_baseline = LGBMRegressor(random_state=42, verbose=-1)

# Huấn luyện mô hình
print("Đang huấn luyện mô hình baseline...")
lgbm_baseline.fit(X_train, y_train)
print("✓ Hoàn thành huấn luyện!")

# Dự đoán trên tập train và validation
y_train_pred_baseline = lgbm_baseline.predict(X_train)
y_val_pred_baseline = lgbm_baseline.predict(X_val)

# Tính toán các metrics
train_mae_baseline = mean_absolute_error(y_train, y_train_pred_baseline)
train_rmse_baseline = np.sqrt(mean_squared_error(y_train, y_train_pred_baseline))
train_r2_baseline = r2_score(y_train, y_train_pred_baseline)

val_mae_baseline = mean_absolute_error(y_val, y_val_pred_baseline)
val_rmse_baseline = np.sqrt(mean_squared_error(y_val, y_val_pred_baseline))
val_r2_baseline = r2_score(y_val, y_val_pred_baseline)

print("="*60)
print("KẾT QUẢ MÔ HÌNH BASELINE (tham số mặc định)")
print("="*60)
print(f"\nTập TRAIN:")
print(f"  MAE:  {train_mae_baseline:.4f} năm")
print(f"  RMSE: {train_rmse_baseline:.4f} năm")
print(f"  R²:   {train_r2_baseline:.4f}")

print(f"\nTập VALIDATION:")
print(f"  MAE:  {val_mae_baseline:.4f} năm")
print(f"  RMSE: {val_rmse_baseline:.4f} năm")
print(f"  R²:   {val_r2_baseline:.4f}")

print(f"\nSố lượng iterations: {lgbm_baseline.n_estimators}")
print(f"Number of leaves: {lgbm_baseline.num_leaves}")
print(f"Learning rate: {lgbm_baseline.learning_rate}")
print("="*60)

# Nhận xét về overfitting
if train_r2_baseline - val_r2_baseline > 0.1:
    print("\n⚠️  Mô hình có dấu hiệu overfitting (R² train >> R² val)")
    print("→  Cần tối ưu hóa siêu tham số để giảm overfitting")
else:
    print("\n✓ Mô hình khá cân bằng, nhưng vẫn có thể cải thiện thêm")

## Bước 5 - Tối ưu hóa siêu tham số với GridSearchCV

Sử dụng GridSearchCV với 5-Fold Cross-Validation để tìm kiếm siêu tham số tối ưu cho mô hình LightGBM.

### Các siêu tham số cần tối ưu:
- **num_leaves:** Số lượng lá tối đa trong một cây (điều khiển độ phức tạp)
- **max_depth:** Độ sâu tối đa của cây (giới hạn overfitting)
- **learning_rate:** Tốc độ học (shrinkage)
- **n_estimators:** Số lượng boosting iterations
- **min_child_samples:** Số mẫu tối thiểu trong một leaf (giống min_samples_leaf)
- **subsample:** Tỷ lệ mẫu được sử dụng (stochastic)
- **colsample_bytree:** Tỷ lệ features được sử dụng cho mỗi cây

In [ ]:
# Định nghĩa không gian siêu tham số để tìm kiếm
param_grid = {
    'num_leaves': [15, 31, 50, 70],
    'max_depth': [5, 7, 10, 15, -1],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

print("="*60)
print("TÌM KIẾM SIÊU THAM SỐ TỐI ƯU")
print("="*60)
print(f"Số lượng tổ hợp tham số: {np.prod([len(v) for v in param_grid.values()])}")
print("Phương pháp: GridSearchCV với 5-Fold Cross-Validation")
print("Metric đánh giá: Negative RMSE (neg_root_mean_squared_error)")
print("\nĐang thực hiện tìm kiếm...")
print("⚠️  Lưu ý: Quá trình này có thể mất thời gian (10-30 phút)")
print("          Tuy nhiên, LightGBM nhanh hơn nhiều so với GBM truyền thống!")
print("="*60)

# Tạo KFold với 5 folds
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Tạo GridSearchCV
grid_search = GridSearchCV(
    estimator=LGBMRegressor(random_state=42, verbose=-1),
    param_grid=param_grid,
    cv=kfold,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)

# Thực hiện tìm kiếm
grid_search.fit(X_train, y_train)

print("\n✓ Hoàn thành tìm kiếm!")

## Bước 6 - Phân tích kết quả GridSearchCV

Hiển thị siêu tham số tối ưu và đánh giá hiệu suất của mô hình tốt nhất.

In [ ]:
# Lấy mô hình tốt nhất và tham số tối ưu
best_lgbm = grid_search.best_estimator_
best_params = grid_search.best_params_
best_cv_score = -grid_search.best_score_  # Chuyển về dương (RMSE)

print("="*60)
print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params.items():
    print(f"  {param:20s}: {value}")

print(f"\nRMSE Cross-Validation (5-fold): {best_cv_score:.4f} năm")
print("="*60)

# Thông tin về mô hình tối ưu
print(f"\nSố lượng iterations: {best_lgbm.n_estimators}")
print(f"Number of leaves: {best_lgbm.num_leaves}")
print(f"Learning rate: {best_lgbm.learning_rate}")
print(f"Max depth: {best_lgbm.max_depth}")

## Bước 7 - Đánh giá mô hình tối ưu trên tập Validation

Sử dụng mô hình đã tối ưu để dự đoán trên tập validation và tính toán các metrics.

In [ ]:
# Dự đoán trên tập train và validation
print("Đang thực hiện dự đoán...")
y_train_pred = best_lgbm.predict(X_train)
y_val_pred = best_lgbm.predict(X_val)

# Tính toán các metrics cho tập train
train_mae = mean_absolute_error(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)

# Tính toán các metrics cho tập validation
val_mae = mean_absolute_error(y_val, y_val_pred)
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
val_r2 = r2_score(y_val, y_val_pred)

print("="*60)
print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print(f"\nTập TRAIN:")
print(f"  MAE:  {train_mae:.4f} năm")
print(f"  RMSE: {train_rmse:.4f} năm")
print(f"  R²:   {train_r2:.4f}")

print(f"\nTập VALIDATION:")
print(f"  MAE:  {val_mae:.4f} năm")
print(f"  RMSE: {val_rmse:.4f} năm")
print(f"  R²:   {val_r2:.4f}")

print("\n" + "="*60)
print("SO SÁNH VỚI BASELINE")
print("="*60)
print(f"Cải thiện RMSE trên validation: {val_rmse_baseline - val_rmse:.4f} năm")
print(f"Cải thiện R² trên validation:   {val_r2 - val_r2_baseline:.4f}")
print("="*60)

## Bước 8 - Phân tích Feature Importance

Phân tích mức độ quan trọng của từng đặc trưng trong việc dự đoán tuổi thọ trung bình. LightGBM cung cấp 2 loại feature importance: split-based và gain-based.

In [ ]:
# Lấy feature importance (gain-based - mặc định)
feature_importance = best_lgbm.feature_importances_

# Tạo DataFrame để sắp xếp
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("="*60)
print("ĐỘ QUAN TRỌNG CỦA CÁC ĐẶC TRƯNG (Feature Importance)")
print("="*60)
for idx, row in importance_df.iterrows():
    print(f"{row['feature']:20s}: {row['importance']:.4f} {'█' * int(row['importance'] / max(importance_df['importance']) * 50)}")
print("="*60)

# Vẽ biểu đồ feature importance
plt.figure(figsize=(10, 6))
colors = plt.cm.viridis(np.linspace(0, 1, len(importance_df)))
plt.barh(importance_df['feature'], importance_df['importance'], color=colors, edgecolor='black')
plt.xlabel('Độ quan trọng (Gain)', fontsize=12, fontweight='bold')
plt.ylabel('Đặc trưng', fontsize=12, fontweight='bold')
plt.title('Feature Importance - LightGBM Regressor', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()

# Tạo thư mục nếu chưa có
os.makedirs('../visualization/lightgbm', exist_ok=True)
plt.savefig('../visualization/lightgbm/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Đã lưu biểu đồ feature importance tại: ../visualization/lightgbm/feature_importance.png")

## Bước 9 - Trực quan hóa kết quả dự đoán

Vẽ biểu đồ so sánh giá trị thực tế và giá trị dự đoán trên tập validation.

In [ ]:
# Tạo figure với 2 subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Subplot 1: Scatter plot - Predicted vs Actual (Validation)
axes[0].scatter(y_val, y_val_pred, alpha=0.5, color='mediumseagreen', edgecolor='black', linewidth=0.5)
axes[0].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2, label='Dự đoán hoàn hảo')
axes[0].set_xlabel('Tuổi thọ thực tế (năm)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Tuổi thọ dự đoán (năm)', fontsize=12, fontweight='bold')
axes[0].set_title(f'Dự đoán vs Thực tế (Validation)\nR² = {val_r2:.4f}, RMSE = {val_rmse:.4f}', 
                  fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Subplot 2: Residual plot
residuals = y_val - y_val_pred
axes[1].scatter(y_val_pred, residuals, alpha=0.5, color='coral', edgecolor='black', linewidth=0.5)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Tuổi thọ dự đoán (năm)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Residuals (Thực tế - Dự đoán)', fontsize=12, fontweight='bold')
axes[1].set_title('Biểu đồ Residuals (Validation)', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../visualization/lightgbm/predictions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Đã lưu biểu đồ dự đoán tại: ../visualization/lightgbm/predictions.png")

## Bước 10 - Phân tích Cross-Validation scores

Trực quan hóa phân bố điểm số RMSE từ 5-Fold Cross-Validation để đánh giá độ ổn định của mô hình.

In [ ]:
# Lấy kết quả cross-validation từ GridSearchCV
cv_results = grid_search.cv_results_
best_index = grid_search.best_index_

# Lấy scores của mô hình tốt nhất qua các folds
best_cv_scores = []
for i in range(5):  # 5 folds
    fold_score = -cv_results[f'split{i}_test_score'][best_index]  # Chuyển về RMSE dương
    best_cv_scores.append(fold_score)

print("="*60)
print("KẾT QUẢ 5-FOLD CROSS-VALIDATION")
print("="*60)
for i, score in enumerate(best_cv_scores, 1):
    print(f"Fold {i}: RMSE = {score:.4f} năm")

print(f"\nTrung bình: {np.mean(best_cv_scores):.4f} năm")
print(f"Độ lệch chuẩn: {np.std(best_cv_scores):.4f} năm")
print(f"Min: {np.min(best_cv_scores):.4f} năm")
print(f"Max: {np.max(best_cv_scores):.4f} năm")
print("="*60)

# Vẽ biểu đồ
plt.figure(figsize=(10, 6))
plt.plot(range(1, 6), best_cv_scores, marker='o', linewidth=2, markersize=10, color='mediumseagreen')
plt.axhline(y=np.mean(best_cv_scores), color='red', linestyle='--', linewidth=2, 
            label=f'Trung bình = {np.mean(best_cv_scores):.4f}')
plt.fill_between(range(1, 6), 
                 np.mean(best_cv_scores) - np.std(best_cv_scores),
                 np.mean(best_cv_scores) + np.std(best_cv_scores),
                 alpha=0.2, color='red', label=f'± 1 std = {np.std(best_cv_scores):.4f}')
plt.xlabel('Fold', fontsize=12, fontweight='bold')
plt.ylabel('RMSE (năm)', fontsize=12, fontweight='bold')
plt.title('RMSE qua các Folds trong Cross-Validation', fontsize=14, fontweight='bold')
plt.xticks(range(1, 6))
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../visualization/lightgbm/cv_scores.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Đã lưu biểu đồ CV scores tại: ../visualization/lightgbm/cv_scores.png")

# Đánh giá độ ổn định
if np.std(best_cv_scores) < 0.5:
    print("\n✓ Mô hình rất ổn định (độ lệch chuẩn < 0.5)")
elif np.std(best_cv_scores) < 1.0:
    print("\n✓ Mô hình khá ổn định (độ lệch chuẩn < 1.0)")
else:
    print("\n⚠️  Mô hình có độ biến động cao (độ lệch chuẩn ≥ 1.0)")

## Bước 11 - Phân tích số lượng Leaves

LightGBM sử dụng leaf-wise tree growth. Phân tích số lượng leaves trong mỗi cây để hiểu độ phức tạp của mô hình.

In [ ]:
# Vẽ biểu đồ về num_leaves tối ưu
print("="*60)
print("PHÂN TÍCH NUM_LEAVES")
print("="*60)
print(f"num_leaves tối ưu: {best_lgbm.num_leaves}")
print(f"max_depth tối ưu: {best_lgbm.max_depth}")

# Tính số leaves tối đa có thể với max_depth
if best_lgbm.max_depth > 0:
    max_possible_leaves = 2 ** best_lgbm.max_depth
    print(f"Số leaves tối đa có thể (với max_depth={best_lgbm.max_depth}): {max_possible_leaves}")
    print(f"Tỷ lệ sử dụng: {best_lgbm.num_leaves / max_possible_leaves * 100:.1f}%")
else:
    print(f"max_depth = -1 (không giới hạn)")

print("\n💡 LƯU Ý:")
print("   - num_leaves > 2^max_depth có thể gây overfitting")
print("   - num_leaves nhỏ → underfitting, mô hình quá đơn giản")
print("   - num_leaves lớn → overfitting, mô hình quá phức tạp")
print("   - Nên giữ num_leaves < 2^max_depth để tránh overfitting")
print("="*60)

# So sánh hiệu suất với các num_leaves khác nhau
num_leaves_values = [15, 31, 50, 70, 100]
num_leaves_results = []

print("\nThử nghiệm với các num_leaves khác nhau:")
for nl in num_leaves_values:
    lgbm_nl = LGBMRegressor(
        num_leaves=nl,
        learning_rate=best_params['learning_rate'],
        n_estimators=100,
        random_state=42,
        verbose=-1
    )
    lgbm_nl.fit(X_train, y_train)
    y_val_pred_nl = lgbm_nl.predict(X_val)
    val_rmse_nl = np.sqrt(mean_squared_error(y_val, y_val_pred_nl))
    num_leaves_results.append(val_rmse_nl)
    print(f"  num_leaves = {nl:3d} → Validation RMSE = {val_rmse_nl:.4f}")

# Vẽ biểu đồ
plt.figure(figsize=(10, 6))
plt.plot(num_leaves_values, num_leaves_results, marker='o', linewidth=2, markersize=10, color='mediumseagreen')
plt.axhline(y=val_rmse, color='red', linestyle='--', linewidth=2, 
            label=f'Best model RMSE = {val_rmse:.4f}')
plt.axvline(x=best_lgbm.num_leaves, color='blue', linestyle='--', linewidth=2, alpha=0.5,
            label=f'Best num_leaves = {best_lgbm.num_leaves}')
plt.xlabel('Number of Leaves', fontsize=12, fontweight='bold')
plt.ylabel('Validation RMSE (năm)', fontsize=12, fontweight='bold')
plt.title('Ảnh hưởng của Number of Leaves đến Hiệu suất', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../visualization/lightgbm/num_leaves_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Đã lưu biểu đồ num_leaves analysis tại: ../visualization/lightgbm/num_leaves_analysis.png")

## Bước 12 - Đánh giá mô hình trên tập Test

Đánh giá hiệu suất cuối cùng của mô hình trên tập test (dữ liệu chưa từng thấy).

In [ ]:
# Dự đoán trên tập test
print("Đang dự đoán trên tập test...")
y_test_pred = best_lgbm.predict(X_test)

# Tính toán các metrics cho tập test
test_mae = mean_absolute_error(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_r2 = r2_score(y_test, y_test_pred)

print("="*60)
print("KẾT QUẢ CUỐI CÙNG TRÊN TẬP TEST")
print("="*60)
print(f"\nMAE:  {test_mae:.4f} năm")
print(f"RMSE: {test_rmse:.4f} năm")
print(f"R²:   {test_r2:.4f}")
print("="*60)

# Tạo bảng so sánh
comparison_df = pd.DataFrame({
    'Tập dữ liệu': ['Train', 'Validation', 'Test'],
    'MAE': [train_mae, val_mae, test_mae],
    'RMSE': [train_rmse, val_rmse, test_rmse],
    'R²': [train_r2, val_r2, test_r2]
})

print("\nBẢNG SO SÁNH KẾT QUẢ:")
print(comparison_df.to_string(index=False))
print("="*60)

# Vẽ biểu đồ so sánh
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ['MAE', 'RMSE', 'R²']
datasets = ['Train', 'Validation', 'Test']
colors = ['#3498db', '#e74c3c', '#2ecc71']

for idx, metric in enumerate(metrics):
    values = comparison_df[metric].values
    axes[idx].bar(datasets, values, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
    axes[idx].set_ylabel(metric, fontsize=12, fontweight='bold')
    axes[idx].set_title(f'{metric} trên các tập dữ liệu', fontsize=13, fontweight='bold')
    axes[idx].grid(True, alpha=0.3, axis='y')
    
    # Thêm giá trị lên đầu cột
    for i, v in enumerate(values):
        axes[idx].text(i, v, f'{v:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('../visualization/lightgbm/comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Đã lưu biểu đồ so sánh tại: ../visualization/lightgbm/comparison.png")

## Bước 13 - Lưu mô hình đã huấn luyện

Lưu mô hình LightGBM đã tối ưu vào file để sử dụng sau này.

In [ ]:
# Tạo thư mục saved_models/lightgbm nếu chưa có
os.makedirs('../saved_models/lightgbm', exist_ok=True)

# Lưu mô hình
model_path = '../saved_models/lightgbm/lightgbm.pkl'
joblib.dump(best_lgbm, model_path)

print("="*60)
print("LƯU MÔ HÌNH")
print("="*60)
print(f"✓ Đã lưu mô hình tại: {model_path}")
print(f"Kích thước file: {os.path.getsize(model_path) / 1024:.2f} KB")
print("="*60)

# Kiểm tra tải lại mô hình
loaded_model = joblib.load(model_path)
y_test_pred_loaded = loaded_model.predict(X_test)
test_rmse_loaded = np.sqrt(mean_squared_error(y_test, y_test_pred_loaded))

print(f"\n✓ Kiểm tra tải lại mô hình:")
print(f"  RMSE trên test (mô hình gốc):      {test_rmse:.4f} năm")
print(f"  RMSE trên test (mô hình đã tải):   {test_rmse_loaded:.4f} năm")
print(f"  Chênh lệch: {abs(test_rmse - test_rmse_loaded):.10f} (rất nhỏ → OK)")

## Bước 14 - Phân tích sâu về Overfitting/Underfitting

Phân tích learning curve để hiểu rõ hơn về hành vi của mô hình.

In [ ]:
from sklearn.model_selection import learning_curve

# Tính toán learning curve
print("Đang tính toán learning curve (có thể mất vài phút)...")
train_sizes, train_scores, val_scores = learning_curve(
    best_lgbm, X_train, y_train,
    cv=5,
    scoring='neg_root_mean_squared_error',
    train_sizes=np.linspace(0.1, 1.0, 10),
    n_jobs=-1,
    random_state=42
)

# Chuyển sang RMSE dương và tính mean, std
train_scores_mean = -train_scores.mean(axis=1)
train_scores_std = train_scores.std(axis=1)
val_scores_mean = -val_scores.mean(axis=1)
val_scores_std = val_scores.std(axis=1)

# Vẽ learning curve
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_scores_mean, 'o-', color='blue', linewidth=2, markersize=8, label='RMSE Train')
plt.plot(train_sizes, val_scores_mean, 'o-', color='red', linewidth=2, markersize=8, label='RMSE Validation')

plt.fill_between(train_sizes, 
                 train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std,
                 alpha=0.2, color='blue')
plt.fill_between(train_sizes,
                 val_scores_mean - val_scores_std,
                 val_scores_mean + val_scores_std,
                 alpha=0.2, color='red')

plt.xlabel('Số lượng mẫu huấn luyện', fontsize=12, fontweight='bold')
plt.ylabel('RMSE (năm)', fontsize=12, fontweight='bold')
plt.title('Learning Curve - LightGBM Regressor', fontsize=14, fontweight='bold')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../visualization/lightgbm/learning_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Đã lưu learning curve tại: ../visualization/lightgbm/learning_curve.png")

# Phân tích
print("\n" + "="*60)
print("PHÂN TÍCH LEARNING CURVE")
print("="*60)
gap = train_scores_mean[-1] - val_scores_mean[-1]
if gap < 0.3:
    print("✓ Khoảng cách giữa train và validation RMSE rất nhỏ")
    print("  → Mô hình cân bằng rất tốt")
elif gap < 0.5:
    print("✓ Khoảng cách giữa train và validation RMSE nhỏ")
    print("  → Mô hình cân bằng tốt")
elif gap < 1.0:
    print("⚠ Khoảng cách giữa train và validation RMSE trung bình")
    print("  → Có dấu hiệu overfitting nhẹ")
else:
    print("✗ Khoảng cách giữa train và validation RMSE lớn")
    print("  → Mô hình bị overfitting")
print("="*60)

## Bước 15 - So sánh tốc độ huấn luyện

So sánh thời gian huấn luyện của LightGBM với các mô hình khác để thấy được lợi thế về tốc độ.

In [ ]:
import time

# So sánh tốc độ với các cấu hình khác nhau
print("="*60)
print("SO SÁNH TỐC ĐỘ HUẤN LUYỆN")
print("="*60)

configs = [
    {'n_estimators': 50, 'num_leaves': 31},
    {'n_estimators': 100, 'num_leaves': 31},
    {'n_estimators': 200, 'num_leaves': 31},
    {'n_estimators': 100, 'num_leaves': 15},
    {'n_estimators': 100, 'num_leaves': 70},
]

results = []
for config in configs:
    lgbm_speed = LGBMRegressor(
        n_estimators=config['n_estimators'],
        num_leaves=config['num_leaves'],
        learning_rate=0.1,
        random_state=42,
        verbose=-1
    )
    
    start_time = time.time()
    lgbm_speed.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    y_val_pred_speed = lgbm_speed.predict(X_val)
    val_rmse_speed = np.sqrt(mean_squared_error(y_val, y_val_pred_speed))
    
    results.append({
        'n_estimators': config['n_estimators'],
        'num_leaves': config['num_leaves'],
        'train_time': train_time,
        'val_rmse': val_rmse_speed
    })
    
    print(f"n_estimators={config['n_estimators']:3d}, num_leaves={config['num_leaves']:2d} → "
          f"Time: {train_time:.3f}s, RMSE: {val_rmse_speed:.4f}")

print("="*60)

# Vẽ biểu đồ
results_df = pd.DataFrame(results)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Thời gian huấn luyện
ax1.bar(range(len(results_df)), results_df['train_time'], color='skyblue', edgecolor='black')
ax1.set_xlabel('Cấu hình', fontsize=12, fontweight='bold')
ax1.set_ylabel('Thời gian (giây)', fontsize=12, fontweight='bold')
ax1.set_title('Thời gian Huấn luyện', fontsize=13, fontweight='bold')
ax1.set_xticks(range(len(results_df)))
ax1.set_xticklabels([f"{r['n_estimators']}/{r['num_leaves']}" for _, r in results_df.iterrows()], rotation=45)
ax1.grid(True, alpha=0.3, axis='y')

# RMSE
ax2.bar(range(len(results_df)), results_df['val_rmse'], color='lightcoral', edgecolor='black')
ax2.set_xlabel('Cấu hình', fontsize=12, fontweight='bold')
ax2.set_ylabel('Validation RMSE (năm)', fontsize=12, fontweight='bold')
ax2.set_title('Hiệu suất Dự đoán', fontsize=13, fontweight='bold')
ax2.set_xticks(range(len(results_df)))
ax2.set_xticklabels([f"{r['n_estimators']}/{r['num_leaves']}" for _, r in results_df.iterrows()], rotation=45)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../visualization/lightgbm/speed_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Đã lưu biểu đồ speed comparison tại: ../visualization/lightgbm/speed_comparison.png")

print("\n💡 KẾT LUẬN:")
print("   - LightGBM cực kỳ nhanh, thậm chí với cấu hình phức tạp")
print("   - Tăng n_estimators → tăng thời gian tuyến tính, cải thiện accuracy")
print("   - Tăng num_leaves → tăng thời gian và accuracy, nhưng dễ overfit")

## Bước 16 - Tạo báo cáo tổng kết

Tổng hợp tất cả thông tin quan trọng về mô hình vào một báo cáo.

In [ ]:
# Tạo báo cáo
report = f"""
{'='*70}
              BÁO CÁO MÔ HÌNH LIGHTGBM REGRESSOR
{'='*70}

1. THÔNG TIN MÔ HÌNH:
   - Loại mô hình: LightGBM Regressor
   - Thư viện: LightGBM (Microsoft)
   - Version: {lgb.__version__}
   - Thời gian tạo: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

2. SIÊU THAM SỐ TỐI ƯU:
   - num_leaves: {best_params['num_leaves']}
   - max_depth: {best_params['max_depth']}
   - learning_rate: {best_params['learning_rate']}
   - n_estimators: {best_params['n_estimators']}
   - min_child_samples: {best_params['min_child_samples']}
   - subsample: {best_params['subsample']}
   - colsample_bytree: {best_params['colsample_bytree']}

3. ĐẶC ĐIỂM MÔ HÌNH:
   - Tree growth strategy: Leaf-wise
   - Số lượng iterations: {best_lgbm.n_estimators}
   - Số đặc trưng: {len(feature_cols)}

4. KẾT QUẢ ĐÁNH GIÁ:
   
   Tập TRAIN:
   - MAE:  {train_mae:.4f} năm
   - RMSE: {train_rmse:.4f} năm
   - R²:   {train_r2:.4f}
   
   Tập VALIDATION:
   - MAE:  {val_mae:.4f} năm
   - RMSE: {val_rmse:.4f} năm
   - R²:   {val_r2:.4f}
   
   Tập TEST (Final):
   - MAE:  {test_mae:.4f} năm
   - RMSE: {test_rmse:.4f} năm
   - R²:   {test_r2:.4f}

5. CROSS-VALIDATION (5-Fold):
   - RMSE trung bình: {np.mean(best_cv_scores):.4f} năm
   - Độ lệch chuẩn: {np.std(best_cv_scores):.4f} năm
   - RMSE tối thiểu: {np.min(best_cv_scores):.4f} năm
   - RMSE tối đa: {np.max(best_cv_scores):.4f} năm

6. SO SÁNH VỚI BASELINE:
   - Cải thiện RMSE validation: {val_rmse_baseline - val_rmse:.4f} năm
   - Cải thiện R² validation: {val_r2 - val_r2_baseline:.4f}

7. TOP 5 ĐẶC TRƯNG QUAN TRỌNG NHẤT:
"""

for i, (idx, row) in enumerate(importance_df.head(5).iterrows(), 1):
    report += f"   {i}. {row['feature']:20s}: {row['importance']:.4f}\n"

report += f"""
8. ƯU ĐIỂM CỦA LIGHTGBM:
   - Tốc độ huấn luyện cực nhanh (nhanh hơn XGBoost 10-20 lần)
   - Hiệu suất cao, thường tương đương hoặc tốt hơn XGBoost
   - Sử dụng bộ nhớ hiệu quả
   - Leaf-wise tree growth cho accuracy cao hơn
   - Hỗ trợ categorical features trực tiếp
   - Hỗ trợ GPU training

9. KỸ THUẬT ĐẶC BIỆT:
   - Gradient-based One-Side Sampling (GOSS)
   - Exclusive Feature Bundling (EFB)
   - Histogram-based algorithm
   - Leaf-wise tree growth thay vì level-wise

10. KẾT LUẬN:
   - Mô hình LightGBM đã được tối ưu hóa thành công
   - Hiệu suất trên tập test: RMSE = {test_rmse:.4f} năm, R² = {test_r2:.4f}
   - LightGBM là lựa chọn tuyệt vời cho dữ liệu lớn và cần tốc độ
   - Mô hình ổn định với độ lệch chuẩn CV: {np.std(best_cv_scores):.4f}
   - Mô hình đã được lưu tại: {model_path}
   - Các biểu đồ đã được lưu tại: ../visualization/lightgbm/

11. LƯU Ý KHI SỬ DỤNG:
   - num_leaves nên < 2^max_depth để tránh overfitting
   - Dữ liệu nhỏ (<1000 mẫu): cẩn thận với overfitting
   - Learning rate nhỏ + n_estimators lớn thường cho kết quả tốt
   - Leaf-wise growth mạnh hơn nhưng dễ overfit hơn level-wise

{'='*70}
"""

print(report)

# Lưu báo cáo vào file
report_path = '../saved_models/lightgbm/lightgbm_report.md'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report)

print(f"\n✓ Đã lưu báo cáo tại: {report_path}")

## Kết luận

### Tổng kết:
1. ✅ **Mô hình LightGBM** đã được xây dựng và tối ưu hóa thành công
2. ✅ **GridSearchCV với 5-Fold CV** đã tìm được bộ siêu tham số tối ưu
3. ✅ **Feature Importance** đã xác định các yếu tố quan trọng nhất
4. ✅ **Hiệu suất cao** với tốc độ huấn luyện cực nhanh
5. ✅ **Mô hình đã được lưu** tại `saved_models/lightgbm/lightgbm.pkl`
6. ✅ **Trực quan hóa chi tiết** đã được tạo tại `visualization/lightgbm/`

### Điểm nổi bật của LightGBM:

#### **1. Tốc độ Vượt trội**
- Nhanh hơn GBM truyền thống 10-100 lần
- Nhanh hơn XGBoost 10-20 lần
- Phù hợp cho dữ liệu lớn và production systems

#### **2. Leaf-wise Tree Growth**
- Tăng accuracy so với level-wise (GBM truyền thống)
- Nhưng dễ overfitting hơn với dữ liệu nhỏ
- Cần điều chỉnh num_leaves và max_depth cẩn thận

#### **3. Kỹ thuật Tối ưu**
- **GOSS:** Giữ gradient lớn, sample gradient nhỏ
- **EFB:** Gộp features sparse
- **Histogram:** Binning thay vì exact split

### So sánh với các thuật toán khác:

| Đặc điểm | GBM | Random Forest | LightGBM |
|----------|-----|---------------|----------|
| Tốc độ | Chậm | Trung bình | Cực nhanh |
| Accuracy | Cao | Cao | Rất cao |
| Overfitting | Trung bình | Thấp | Cao (với dữ liệu nhỏ) |
| Bộ nhớ | Trung bình | Cao | Thấp |
| Tree growth | Level-wise | Parallel | Leaf-wise |

### Khi nào nên dùng LightGBM:

✅ **NÊN DÙNG:**
- Dữ liệu lớn (>10,000 mẫu)
- Cần tốc độ huấn luyện nhanh
- Cần hiệu suất dự đoán cao nhất
- Có nhiều features
- Production systems cần inference nhanh

❌ **KHÔNG NÊN DÙNG:**
- Dữ liệu quá nhỏ (<1000 mẫu) → dễ overfit
- Cần giải thích chi tiết từng quyết định
- Không có thời gian tune hyperparameters
- Môi trường bị giới hạn bộ nhớ rất nghiêm ngặt

### Tips để tránh Overfitting:

1. **Giảm num_leaves**: Bắt đầu với 31, giảm xuống nếu overfit
2. **Tăng min_child_samples**: Tăng từ 20 lên 50-100
3. **Giảm max_depth**: Giới hạn độ sâu cây
4. **Tăng regularization**: Sử dụng `reg_alpha`, `reg_lambda`
5. **Subsample & colsample**: Dùng 0.7-0.9 thay vì 1.0
6. **Learning rate nhỏ**: 0.01-0.05 với nhiều iterations

### Bước tiếp theo:
- So sánh chi tiết với XGBoost
- Thử nghiệm với CatBoost
- Xây dựng ensemble model kết hợp tất cả boosting algorithms
- Fine-tuning thêm với Bayesian Optimization
- Deployment vào production với ONNX hoặc serving frameworks